# notion to todoist development notebook

In [1]:
import os
import sys
import notion
from notion.client import NotionClient
from todoist.api import TodoistAPI

sys.path.insert(1, '/Users/jarredgreen/PycharmProjects/notion-to-doist/notion2doist')
from app import (
    task, syncManager, labelManager, 
    projectManager, taskManager, helper, compute_hmac, check_notion_for_updates,
    webhook_arrived
)
from bidict import bidict
from datetime import datetime, timezone, timedelta

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
todoist_token = "1aa330c1397a63ce19baba1c3401c42af9bbaa08"
notion_token = "a2ae36c266053a72fc564b418a1f578d736228f614ab5925be62f22383894f550dd53004bb48005e699e101a94e00008d23fd6eff24286a48f62f943d3e427ddb14092ffcf94df25e58dd939bfc8"
notion_settings = "https://www.notion.so/jarredgreen/c1c3fea02d4c4a52bfece58af9c407c3?v=ad00b0ae035344caa12f664c7efe7d77"

In [75]:
man = syncManager(
    todoist_token=todoist_token,
    notion_token=notion_token,
    notion_settings_url=notion_settings
)

Sync configuration updated.


In [80]:
man.sync_todoist_api()
for item in man.api.items.all():
    #print(item["content"], item["id"], item["checked"])
    item.delete()

In [55]:
for label in man.api.labels.all():
    label.delete()

In [57]:
for project in man.api.projects.all():
    project.delete()

In [81]:
man.commit_todoist_api()

## Testing sync functions

In [130]:
man.sync_config()

Sync configuration updated.


In [14]:
tools.labelManager.sync_labels_to_todoist(man)

Syncing labels...
Done syncing labels. 0 label(s) added and 0 label(s) updated


In [52]:
tools.projectManager.sync_projects(man)

Syncing projects...
Adding notion project ID for task: Project 1
Adding notion project ID for task: Project 2
Adding notion project ID for task: Project 3
Done syncing projects. 0 project(s) added and 0 projects(s) updated


Task G ID: 4251791466

In [80]:
tools.taskManager.sync_notion_to_todoist(man)

Syncing notion tasks to todoist...
Task: "Task D": no change
Task: "Task F": no change
Task: "Task E"
    label_ids: t̶o̶d̶o̶i̶s̶t̶:̶ ̶[̶2̶1̶5̶5̶3̶3̶5̶5̶5̶0̶,̶ ̶2̶1̶5̶5̶3̶0̶5̶7̶3̶1̶]̶ ⮕ notion: [2155335550, 2155305731, 2155305730]
    label_names: t̶o̶d̶o̶i̶s̶t̶:̶ ̶[̶'̶W̶a̶i̶t̶i̶n̶g̶'̶,̶ ̶'̶q̶u̶i̶c̶k̶'̶]̶ ⮕ notion: ['Waiting', 'quick', 'email']
Task: "Task B": no change
Task: "Task C": no change
Task: "Task A": no change
Added 0 and updated 1 tasks.


In [47]:
taskg = man.api.items.get_by_id(4251791466)
itemg = tools.taskManager.from_todoist(man, taskg)
rowg = man.tasks.collection.get_rows(search="Task G")[0]
notiong = tools.taskManager.from_notion(man, rowg)

In [79]:
taskManager.webhook_arrived(man, {"event_name": "item:updated"})

current item id: 4281201523
Item 4281201523 was just synced from notion. Skipping...
Updating task "Hi is okay":
    content: n̶o̶t̶i̶o̶n̶:̶ ̶H̶i̶ ̶i̶s̶ ̶o̶k̶ ̶p̶3̶ ⮕ todoist: Hi is okay
    project_id: n̶o̶t̶i̶o̶n̶:̶ ̶2̶2̶4̶6̶0̶6̶2̶9̶6̶2̶ ⮕ todoist: 2248315288
    project_name: n̶o̶t̶i̶o̶n̶:̶ ̶I̶n̶b̶o̶x̶ ⮕ todoist: Project 1
    notion_project_id: n̶o̶t̶i̶o̶n̶:̶ ̶8̶7̶1̶c̶9̶3̶6̶e̶-̶a̶e̶f̶7̶-̶4̶c̶e̶b̶-̶8̶c̶9̶2̶-̶7̶6̶7̶6̶3̶e̶6̶7̶d̶e̶e̶b̶ ⮕ todoist: 043094db-bc90-4d4e-a02a-a5ae01abd937
Updating title from Hi is ok p3 to Hi is okay


## Current tests

### why is everything broken

In [44]:
items = []
for row in man.tasks.collection.get_rows():
    row.refresh()
    items.append(taskManager.from_notion(man, row))

In [18]:
projectManager.sync_projects(man)

Syncing projects...
Adding notion project ID to: Inbox
Done syncing projects. 0 project(s) added and 0 projects(s) updated


In [6]:
labelManager.sync_labels_to_todoist(man)

Syncing labels...
Done syncing labels. 0 label(s) added and 0 label(s) updated


In [54]:
datetime.fromtimestamp(float(1577833200.5))

datetime.datetime(2020, 1, 1, 0, 0, 0, 500000)

In [59]:
man.config

{'Label column name': {'status': 'select', 'tags': 'multi_select'},
 'Link to task database': 'jarredgreen/f7038e6d4ffb4118aed5d5e722e6e862?v=d3ece62f73814958bc1e07e0abd241e6',
 'Link to project database': 'jarredgreen/f817aed3992c4acfa8f2889cd0159611?v=35665a3e284f4da0a36451303b9b2478',
 'Link to label database': 'jarredgreen/46fbda888f844ca9a429ccc7bfff35dc?v=8a7729ac1dce4347883e49d1cbd77eac',
 'Sync completed tasks': False,
 'Use groups': False,
 'Sync password': 'ilovecats',
 'Last notion sync time': '1577833200.0',
 'Input label string': 'status=select, tags=multi_select'}

In [76]:
taskManager.sync_notion_to_todoist(man, full_sync=True)

Syncing notion tasks to todoist...



---------------------------------

 Notion task: Task: Hi is ok p3
 {
    "source": "notion",
    "task_id": 4281201523,
    "content": "Hi is ok p3",
    "done": false,
    "due": "2020-10-28",
    "label_ids": [
        2155428351,
        2155428356
    ],
    "label_names": [
        "Todo",
        "quick"
    ],
    "project_id": 2246062962,
    "project_name": "Inbox",
    "notion_task_id": "c0f119a3-d7e9-4eeb-b373-286dd999cc50",
    "notion_project_id": "871c936e-aef7-4ceb-8c92-76763e67deeb",
    "todoist_note_id": null
}
Task: "Hi is ok p3"
 Todoist task: Task: Hi is ok!
 {
    "source": "todoist",
    "task_id": 4281201523,
    "content": "Hi is ok!",
    "done": false,
    "due": "2020-10-28",
    "label_ids": [
        2155428351,
        2155428356
    ],
    "label_names": [
        "Todo",
        "quick"
    ],
    "project_id": 2246062962,
    "project_name": "Inbox",
    "notion_task_id": "c0f119a3-d7e9-4eeb-b373-286dd999cc50",
  

In [78]:
sorted(['Todo', 'Quick', 'email'])

['Quick', 'Todo', 'email']

In [72]:
man.update_notion_sync_time()

In [73]:
man.last_notion_sync_time

datetime.datetime(2020, 10, 24, 16, 26, 12, 839307)

In [61]:
man.settings.collection.get_rows(search="Last notion sync time")

[
  <CollectionRowBlock (id='60a2dc47-35d4-47a1-b59c-30e70a54afdf', title='Last notion sync time')>,
]

### encrpytion

In [22]:
import base64
import hashlib
import hmac

In [48]:
def compute_hmac(body):

    signature = base64.b64encode(hmac.new(
        "secretpassword".encode('utf-8'),
        body.encode('utf-8'),
        digestmod=hashlib.sha256).digest()).decode('utf-8')
    return signature

In [57]:
compute_hmac("test")

'aBDj5zLaMcPbTGSYGXwErGT3Cq9zRX4QoKRet+YrE30='